# Experiment: Comparing different devices for training

In [1]:
using Revise, QARBoM, DWave, DataFrames, CSV, QiskitOpt, MQLib, PySA

Precompiling QARBoM
  ✓ QARBoM
  1 dependency successfully precompiled in 7 seconds. 105 already precompiled.


In [9]:
MOI = QARBoM.QUBO.ToQUBO.MOI
learning_rate = 0.0001
# num_sweeps = [i for i in 1:25:101]
# num_reads = [i for i in 1:25:101]
W = randn(22,10)

22×10 Matrix{Float64}:
  0.00732249  -0.597281    -1.14158    …  -0.827658   -0.280026    0.691945
 -0.389432     0.62191     -0.122262      -1.108       2.13116    -0.250495
  0.0910674   -0.115599    -1.06342       -0.853588    0.877599    0.702235
  1.18809     -0.201831    -1.72035       -1.26608    -1.3339     -2.43712
  0.184519     1.27885     -0.425197      -0.855397    1.11735     1.32804
  0.769346     0.040552     0.51079    …   0.325429   -2.22408     0.446216
  0.359262     0.129737     0.930199       0.0412884  -0.736087   -0.739737
  1.14716      0.454543     0.651143       0.19507    -0.818735   -1.53663
 -0.303722     0.00443448  -2.01943       -0.650412    0.594495    0.703685
 -0.841654    -2.35914      0.25134        0.428629   -0.0907988   0.960686
 -1.34785     -0.190396     0.679303   …  -1.31006    -0.751432    0.327842
  1.49373      2.16001      2.08856        0.380743    0.266644   -0.669622
  1.64706      0.364672     1.27454        0.294491    1.42674     1

In [10]:
df = DataFrame(CSV.File(raw"./converted_bool_only.csv"))

x_train = Vector{Vector{Int}}()

for row in eachrow(df)
    push!(x_train, collect(row))
end

# DWave

In [55]:
rbm_dwave_real = RBM(22,10, W)

RBM([0.007322489037212931 -0.5972807707650185 … -0.28002615835011785 0.6919447529442866; -0.3894315397644169 0.6219102872316089 … 2.1311557671818466 -0.2504947307552616; … ; 0.019479846972800304 0.997515211317619 … 0.7751129695207407 -0.067013580207158; -1.0878580114435195 -0.11858713450143928 … -1.394071939838351 -0.4789131843422072], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 22, 10)

In [56]:
MOI.supports(::DWave.Optimizer, ::MOI.ObjectiveSense) = true
function setup_dwave_real(model, sampler)
    MOI.set(model, MOI.RawOptimizerAttribute("num_reads"), 25)
end

setup_dwave_real (generic function with 1 method)

In [57]:
mse_dwave_real = QARBoM.train_persistent_qubo!(
    rbm_dwave_real, 
    x_train[1:10000];
    batch_size = 10, 
    n_epochs = 50,  
    learning_rate = [learning_rate for i in 1:50],
    model_setup = setup_dwave_real,
    sampler = DWave.Optimizer
)

Setting up QUBO model
Setting mini-batches
Starting training


julia(24880) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(24881) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(24882) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(24886) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


LoadError: Python: SolverFailureError: Problem not accepted because user has insufficient remaining solver access time in project DEV

# DWave.Neal

In [11]:
rbm_dwave = RBM(22,10, W)

RBM([0.007322489037212931 -0.5972807707650185 … -0.28002615835011785 0.6919447529442866; -0.3894315397644169 0.6219102872316089 … 2.1311557671818466 -0.2504947307552616; … ; 0.019479846972800304 0.997515211317619 … 0.7751129695207407 -0.067013580207158; -1.0878580114435195 -0.11858713450143928 … -1.394071939838351 -0.4789131843422072], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 22, 10)

In [12]:
MOI.supports(::DWave.Neal.Optimizer, ::MOI.ObjectiveSense) = true
function setup_dwave(model, sampler)
    MOI.set(model, MOI.RawOptimizerAttribute("num_reads"), 25)
    MOI.set(model, MOI.RawOptimizerAttribute("num_sweeps"), 100)
end

setup_dwave (generic function with 1 method)

In [13]:
mse_dwave = QARBoM.train_persistent_qubo!(
    rbm_dwave, 
    x_train[1:10000];
    batch_size = 10, 
    n_epochs = 50,  
    learning_rate = [learning_rate for i in 1:50],
    model_setup = setup_dwave,
    sampler = DWave.Neal.Optimizer
)

Setting up QUBO model
Setting mini-batches
Starting training
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     1 |    6.2581 |        0.0139 |      12.2502 |        1.9828 |   14.2468 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     2 |    5.5839 |        0.0125 |       4.6838 |        1.8587 |   20.8018 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsam

50-element Vector{Float64}:
 6.258103516529083
 5.583884359684325
 4.9985374612605336
 4.477669597731513
 4.035836503550631
 3.700132070686311
 3.440381873478058
 3.245028743297952
 3.0882479192566357
 2.9608663888226636
 2.8558805711447968
 2.767346257596144
 2.691005512438376
 ⋮
 2.1039538559127084
 2.0974032551787487
 2.0913427474212836
 2.0857020046921413
 2.0805836647209306
 2.0761777200089386
 2.072260599943042
 2.06887972408469
 2.065595150980985
 2.062519468681566
 2.059263286436603
 2.0564079359743235

# Qiskit

In [ ]:
rbm_qiskit = RBM(22,10, W)

In [ ]:
MOI.supports(::QAOA.Optimizer, ::MOI.ObjectiveSense) = true
function setup_qiskit(model, sampler)
    MOI.set(rbm.model, QAOA.NumberOfReads(), 1000)
    MOI.set(rbm.model, QAOA.MaximumIterations(), 100)
    MOI.set(rbm.model, QAOA.Channel(), "ibm_quantum") # or QAOA.Instance
    MOI.set(rbm.model, QAOA.Instance(), "ibm-q-asu/main/purdue-david-ber") # or QAOA.Instance
    MOI.set(rbm.model, QAOA.IBMBackend(), "ibm_kyiv") # or QAOA.IBMFakeBackend
end

In [8]:
mse_qiskit = QARBoM.train_persistent_qubo!(
    rbm_qiskit, 
    x_train[1:10000];
    batch_size = 10, 
    n_epochs = 50,  
    learning_rate = [learning_rate for i in 1:50],
    model_setup = setup_qiskit,
    sampler = QAOA.Optimizer
)

Setting mini-batches
Starting training
Running QAOA on ibm_kyiv...


LoadError: InterruptException:

# MQLib

In [45]:
rbm_mqlib = RBM(22,10, W)

RBM([0.007322489037212931 -0.5972807707650185 … -0.28002615835011785 0.6919447529442866; -0.3894315397644169 0.6219102872316089 … 2.1311557671818466 -0.2504947307552616; … ; 0.019479846972800304 0.997515211317619 … 0.7751129695207407 -0.067013580207158; -1.0878580114435195 -0.11858713450143928 … -1.394071939838351 -0.4789131843422072], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 22, 10)

In [50]:
println(keys(MQLib._HEURISTICS))

["FESTA2002GVNSPR", "LAGUNA2009CE", "MERZ1999CROSS", "FESTA2002GVNS", "BEASLEY1998SA", "PALUBECKIS2004bMST1", "DESOUSA2013", "BASELINE", "GLOVER1998a", "BURER2002", "DUARTE2005", "KATAYAMA2001", "MERZ1999GLS", "HASAN2000GA", "FESTA2002GPR", "MERZ2002ONEOPT", "MERZ2004", "GLOVER2010", "PALUBECKIS2004bMST5", "PALUBECKIS2004bMST3", "LU2010", "ALKHAMIS1998", "MERZ1999MUTATE", "MERZ2002GREEDY", "MERZ2002GREEDYKOPT", "PALUBECKIS2004bSTS", "PALUBECKIS2004bMST2", "FESTA2002G", "LAGUNA2009HCE", "MERZ2002KOPT", "PARDALOS2008", "FESTA2002VNS", "BEASLEY1998TS", "LODI1999", "HASAN2000TS", "PALUBECKIS2004bMST4", "FESTA2002VNSPR", "KATAYAMA2000", "PALUBECKIS2006"]


In [51]:
MOI.supports(::MQLib.Optimizer, ::MOI.ObjectiveSense) = true
function setup_mqlib(model, sampler)
    QARBoM.JuMP.set_silent(model)
    MOI.set(model, MQLib.Heuristic(),  "BURER2002")
end

setup_mqlib (generic function with 1 method)

In [52]:
mse_mqlib = QARBoM.train_persistent_qubo!(
    rbm_mqlib, 
    x_train[1:10000];
    batch_size = 10, 
    n_epochs = 50,  
    learning_rate = [learning_rate for i in 1:50],
    model_setup = setup_mqlib,
    sampler = MQLib.Optimizer
)

Setting up QUBO model
Setting mini-batches
Starting training
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     1 |    6.2914 |        0.1920 |    1016.5300 |        3.1668 | 1019.8888 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     2 |    5.6604 |        0.3082 |    1020.2263 |        3.4892 | 2043.9125 |
|------------------------------------------------------------------------------|


LoadError: InterruptException:

# PySA

In [17]:
rbm_pysa = RBM(22,10, W)

RBM([0.007322489037212931 -0.5972807707650185 … -0.28002615835011785 0.6919447529442866; -0.3894315397644169 0.6219102872316089 … 2.1311557671818466 -0.2504947307552616; … ; 0.019479846972800304 0.997515211317619 … 0.7751129695207407 -0.067013580207158; -1.0878580114435195 -0.11858713450143928 … -1.394071939838351 -0.4789131843422072], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 22, 10)

In [18]:
MOI.supports(::PySA.Optimizer, ::MOI.ObjectiveSense) = true
function setup_pysa(model, sampler) 
    QARBoM.JuMP.set_silent(model)    
    MOI.set(model, PySA.NumberOfSweeps(), 5)
end

setup_pysa (generic function with 1 method)

In [19]:
mse_pysa = QARBoM.train_persistent_qubo!(
    rbm_pysa, 
    x_train[1:10000];
    batch_size = 10, 
    n_epochs = 50,  
    learning_rate = [learning_rate for i in 1:50],
    model_setup = setup_pysa,
    sampler = PySA.Optimizer
)

Setting up QUBO model
Setting mini-batches
Starting training
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     1 |    6.3288 |        0.0290 |      40.0825 |        2.9624 |   43.0739 |
|------------------------------------------------------------------------------|


julia(17836) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/pripper/Documents/GitHub/RBM/example/.CondaPkg/env/lib/python3.11/site-packages/pysa/simulation.py:155: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'C', False, aligned=True))
  energies[k] += sweep(update_spin, couplings, local_fields,
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     2 |    5.7857 |        0.0175 |      29.3031 |        2.7062 |   75.1006 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     3 |    5.2911 |        0.0178 |      30.1208 |        2.7093 |  107.9486 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Qsamp) | Time (Update) | Total     |
|---------------------------

50-element Vector{Float64}:
 6.328806840100925
 5.785687647837738
 5.291058395336578
 4.838723720619781
 4.431966531043428
 4.0629997208182544
 3.736497699101911
 3.4544759684119635
 3.2169845267074852
 3.0201699348589477
 2.8657887243131097
 2.7388463913457506
 2.634909307083326
 ⋮
 2.1308855741034267
 2.130643441771649
 2.1300490937162215
 2.130139906638105
 2.1303057104275087
 2.1309429157391087
 2.131385976535845
 2.132145954105497
 2.133068449208464
 2.1347562718523894
 2.1359007995019303
 2.1370192742794014

In [13]:
df_pysa = DataFrame(:mse => mse_pysa)
CSV.write("mse_pysa.csv", df_pysa)

"mse_pysa.csv"

# Classical PCD

In [20]:
rbm_pcd = RBM(22,10, W)

RBM([0.007322489037212931 -0.5972807707650185 … -0.28002615835011785 0.6919447529442866; -0.3894315397644169 0.6219102872316089 … 2.1311557671818466 -0.2504947307552616; … ; 0.019479846972800304 0.997515211317619 … 0.7751129695207407 -0.067013580207158; -1.0878580114435195 -0.11858713450143928 … -1.394071939838351 -0.4789131843422072], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 22, 10)

In [21]:
mse_pcd = QARBoM.train_pcd!(rbm_pcd, x_train[1:10000]; n_epochs = 50, batch_size = 10, learning_rate = learning_rate)

Setting mini-batches
Starting training
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Gibbs) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     1 |    6.2817 |        0.0431 |       0.0708 |        0.0433 |    0.1573 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Gibbs) | Time (Update) | Total     |
|------------------------------------------------------------------------------|
|     2 |    5.6452 |        0.0185 |       0.1102 |        0.0195 |    0.3055 |
|------------------------------------------------------------------------------|
|------------------------------------------------------------------------------|
| Epoch |    MSE    | Time (Sample) | Time (Gibbs) | Time (Update) | T

50-element Vector{Float64}:
 6.281708295777717
 5.645208482743743
 5.0724907656772364
 4.563038960257008
 4.12690626292462
 3.7672850070088635
 3.476527223012211
 3.2423949229419637
 3.057025201906265
 2.9064752246300696
 2.7840470273966234
 2.6822468812704447
 2.596114858896404
 ⋮
 1.9801220291217185
 1.9762874637402659
 1.9747079833539454
 1.9730526313526018
 1.971127654457904
 1.9693416445195533
 1.9681228136844606
 1.9664432510556236
 1.9654851290057196
 1.9653271124024478
 1.9646341920827708
 1.9635835312369208

In [22]:
using Plots

In [23]:
df = DataFrame(
    "mse_dwave" => mse_dwave,
    "mse_pysa" => mse_pysa,
    "mse_pcd" => mse_pcd)

Row,mse_dwave,mse_pysa,mse_pcd
,Float64,Float64,Float64
1,6.2581,6.32881,6.28171
2,5.58388,5.78569,5.64521
3,4.99854,5.29106,5.07249
4,4.47767,4.83872,4.56304
5,4.03584,4.43197,4.12691
6,3.70013,4.063,3.76729
7,3.44038,3.7365,3.47653
8,3.24503,3.45448,3.24239
9,3.08825,3.21698,3.05703


In [24]:
CSV.write("mse_all.csv", df)

"mse_all.csv"

In [25]:
df_loaded = DataFrame(CSV.File("mse_all.csv"))

Row,mse_dwave,mse_pysa,mse_pcd
,Float64,Float64,Float64
1,6.2581,6.32881,6.28171
2,5.58388,5.78569,5.64521
3,4.99854,5.29106,5.07249
4,4.47767,4.83872,4.56304
5,4.03584,4.43197,4.12691
6,3.70013,4.063,3.76729
7,3.44038,3.7365,3.47653
8,3.24503,3.45448,3.24239
9,3.08825,3.21698,3.05703


In [42]:
plot(Matrix(df_loaded), labels=permutedims(names(df_loaded)), linewidth=2)
theme(:wong)
lens!([49, 50], [1.88, 2.5], inset = (1, bbox(0.5, 0.0, 0.4, 0.4)), subplot=2)

plot!(legend=:outerbottom, legendcolumns=3)

xlabel!("Epoch", subplot =1)
ylabel!("Mean Squared Error", subplot =1 )

savefig("benchmark_devices.pdf")

"/Users/pripper/Documents/GitHub/RBM/example/Heart/benchmark_devices.pdf"